In [ ]:
# !pip install pretrainedmodels==0.7.4
# !pip install efficientnet-pytorch==0.6.3
# !pip install timm==0.3.2

In [ ]:
# # 1.6.0 버전
# !pip install torch==1.6.0
# !pip install torchvision==0.7.0

In [1]:
!nvidia-smi

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import SimpleITK as sitk
import cv2
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('/workspace/sunggu'))
sys.path.append(os.path.abspath('/workspace/sunggu/4.Dose_img2img/utils'))
sys.path.append(os.path.abspath('/workspace/sunggu/MONAI'))
from sunggu_utils import check_value, take_list, plot_confusion_matrix, list_sort_nicely, find_dir, plot_3D

import piq

Thu May 13 02:40:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:02:00.0 Off |                  N/A |
| 41%   46C    P2   116W / 280W |   5982MiB / 24220MiB |     21%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 42%   

In [2]:
torch.__version__

'1.6.0'

## Setup imports

In [3]:
import glob
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
import torch
import skimage
import monai

from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import CacheDataset, DataLoader, Dataset
from monai.inferers import sliding_window_inference
from monai.losses import DiceLoss, GeneralizedDiceLoss, FocalLoss, TverskyLoss, NoiseRobustDiceLoss
from monai.metrics import compute_meandice, DiceMetric, ConfusionMatrixMetric 
from monai.networks.layers import Norm
from monai.networks.nets import UNet, highresnet
from monai.transforms import (
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadNiftid,
    LoadNumpyd,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    Lambdad,
    ToTensord,
    RandScaleIntensityd,
    RandGaussianNoised,
    RandFlipd,
    RandZoomd,
    RandGaussianNoised,
    RandGaussianSmoothd,
    RandShiftIntensityd,
    SpatialPadd,
    RandAffined,
    CastToTyped,
    DeleteItemsd,
    FgBgToIndicesd,
    Rand3DElasticd,
    RandZoomd,
    Rand2DElasticd,
    RandWeightedCropd,
    AsDiscrete,
    SpatialPadd,
    adaptor,
)
from monai.utils import first, set_determinism

print_config()

MONAI version: 0+unknown
Python version: 3.6.9 (default, Oct  8 2020, 12:12:24)  [GCC 8.4.0]
OS version: Linux (4.4.0-206-generic)
Numpy version: 1.19.5
Pytorch version: 1.6.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.2.1
scikit-image version: 0.17.2
Pillow version: 8.1.0
Tensorboard version: 2.4.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.7.0
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.56.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Set 시드

In [4]:
import random

random_seed = 7
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

set_determinism(seed=7)

## Set Train / Test

In [ ]:
test_low_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/2D_dataset/Test/*/*Low Dose Neck*/*.npy"))
test_high_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/2D_dataset/Test/*/*Neck Other*/*.npy"))

dcm_test_low_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Test/*/*Low Dose Neck*/*/*.dcm"))
dcm_test_high_images = list_sort_nicely(glob.glob("/workspace/sunggu/4.Dose_img2img/dataset/Cleansing_dcm_dataset/Test/*/*Neck Other*/*/*.dcm"))

test_files = [{"low": low_name, "high": high_name, "dcm_low" : dcm_low, "dcm_high" : dcm_high} for low_name, high_name, dcm_low, dcm_high in zip(test_low_images,
                                                                                  test_high_images, 
                                                                                  dcm_test_low_images, 
                                                                                  dcm_test_high_images)]

## Augmentation

In [5]:
# CT에 맞는 Augmentation
test_transforms = Compose(
    [
        LoadNumpyd(keys=["low", "high"]),
        AddChanneld(keys=["low", "high"]),   
        ToTensord(keys=["low", "high"]),
    ]
)

## Check transforms in DataLoader

In [6]:
check_ds = Dataset(data=test_files, transform=test_transforms)
# check_loader = DataLoader(check_ds, batch_size=1, shuffle=False)
# check_data = next(iter(check_loader))

check_data = check_ds[300]

print(check_data['low_meta_dict']['filename_or_obj'])
print(check_data['high_meta_dict']['filename_or_obj'])

low = (check_data["low"][0])
high = (check_data["high"][0])
print(f"image shape: {low.shape}")

plt.figure("check", (12, 6))
plt.subplot(1, 2, 1)
plt.title("low")
plt.imshow(low, cmap="gray")

plt.subplot(1, 2, 2)
plt.title("high")
plt.imshow(high, cmap="gray")

plt.show()

NameError: name 'test_files' is not defined

In [ ]:
import multiprocessing
print("CPU 갯수 = ", multiprocessing.cpu_count())

# Cachedataset 이거 뭔가 문제가 있음...
test_ds = Dataset(data=test_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, num_workers=16, drop_last=False)

## Create Model, Loss, Optimizer

In [ ]:
# def lambda_rule(epoch):
#     lr_l = 1.0 - max(0, epoch + 1 - 100) / float(100 + 1)
#     return lr_l

# scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_rule)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, threshold=0.01, patience=5)
# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)

In [7]:
from Cyclegan_sunggu.model import *
from Cyclegan_sunggu.util import *
import itertools

device = 'cuda'

## 모델
netG_low_2_high = CycleGAN_Generator(in_channels=1, out_channels=1, feature=64, norm='inorm', nblk=9)
netG_high_2_low = CycleGAN_Generator(in_channels=1, out_channels=1, feature=64, norm='inorm', nblk=9)

netD_low  = CycleGAN_Discriminator(in_channels=1, out_channels=1, feature=64, norm='inorm')
netD_high = CycleGAN_Discriminator(in_channels=1, out_channels=1, feature=64, norm='inorm')

## 모델 이어서 학습하기

In [ ]:
## 네트워크 불러오기
checkpoint_dir = '/workspace/sunggu/4.Dose_img2img/model/Cycle_Gan_2D/epoch_215_model.pth'
checkpoint = torch.load(checkpoint_dir)

epoch = checkpoint['epoch']

netG_low_2_high.load_state_dict(checkpoint['netG_low_2_high_state_dict'])
netG_high_2_low.load_state_dict(checkpoint['netG_high_2_low_state_dict'])

netD_low.load_state_dict(checkpoint['netD_low_state_dict'])
netD_high.load_state_dict(checkpoint['netD_high_state_dict'])

# print(checkpoint['optimizer_G_state_dict'])
# print(checkpoint['optimizer_D_state_dict'])


In [ ]:
netG_low_2_high.to('cuda')
netG_high_2_low.to('cuda')
netD_low.to('cuda')
netD_high.to('cuda')

###### 필요한 Weight만 Load

In [ ]:
# # Before
# model_dict = model.state_dict()
# print("이전 weight = ", model_dict['encoder._conv_stem.weight'][0])

# load_dir = '/workspace/sunggu/1.Hemorrhage/monai_experiment/model/Efficient3d_conv2d_Aux/'
# pretrained_dict =  torch.load(os.path.join(load_dir, "epoch_0_best_metric_model.pth")) 

# # 1. filter out unnecessary keys
# pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# # 2. overwrite entries in the existing state dict
# model_dict.update(pretrained_dict) 
# # 3. load the new state dict
# model.load_state_dict(model_dict)

# # After
# print("이후 weight = ", model_dict['encoder._conv_stem.weight'][0])

## TEST

In [ ]:
## 그밖에 부수적인 functions 설정하기
fn_tonumpy = lambda x: x.cpu().detach().numpy().transpose(0, 2, 3, 1)
fn_denorm = lambda x: (x * STD) + MEAN

In [ ]:
def return_preprocessing(x):
    x = (4096.0)*x - 1024.0
    return x

def return_preprocessing_for_dcm(x):
    x = (4096.0)*x
    return x

In [ ]:
import numpy as np
from pydicom import dcmread

def save_dicom(original_dcm_path, np_img, save_path):
    dcm = dcmread(original_dcm_path)
#     dcm.is_little_endian = True
#     dcm.is_implicit_VR = False
#     dcm.PixelData = np_img.astype('uint16').squeeze().tostring()
    dcm.PixelData = np_img.astype('int16').squeeze().tobytes()
    
    dcm.save_as(save_path)
    print(save_path)

In [ ]:
epoch= 37
test_low2high_save_folder = '/workspace/sunggu/4.Dose_img2img/Predictions/dcm/test_cyclegan_2d_epoch_'+str(epoch) + '/low2high/'
test_high2low_save_folder = '/workspace/sunggu/4.Dose_img2img/Predictions/dcm/test_cyclegan_2d_epoch_'+str(epoch) + '/high2low/'

with torch.no_grad():
    # Model 선언
    netG_low_2_high.eval()
    netG_high_2_low.eval()
    netD_low.eval()
    netD_high.eval()

    test_iterator = tqdm(test_loader, desc='Test', file=sys.stdout)    
    for batch_data in test_iterator:

        os.makedirs(test_low2high_save_folder+batch_data['dcm_low'][0].split('/')[7], mode=0o777, exist_ok=True)
        os.makedirs(test_high2low_save_folder+batch_data['dcm_high'][0].split('/')[7], mode=0o777, exist_ok=True)
        
        # forward pass
        input_low = batch_data['low'].to(device)
        input_high = batch_data['high'].to(device)
        
        # Forward Generator
        output_high = netG_low_2_high(input_low)
        output_low = netG_high_2_low(input_high)

        # 저장하기
#         print(input_low.min(), input_low.max())
        input_low = fn_tonumpy(return_preprocessing_for_dcm(input_low))
        print(input_low.shape)
        input_low[:, 128:128+256, 128:128+256, :] = 0
        input_high = fn_tonumpy(return_preprocessing_for_dcm(input_high))
        output_high = fn_tonumpy(return_preprocessing_for_dcm(output_high))
        output_low = fn_tonumpy(return_preprocessing_for_dcm(output_low))
        
        save_dicom(batch_data['dcm_low'][0], input_low, test_low2high_save_folder+batch_data['dcm_low'][0].split('/')[7]+'/gt_low_' +batch_data['dcm_low'][0].split('/')[-1])
        save_dicom(batch_data['dcm_low'][0], output_high, test_low2high_save_folder+batch_data['dcm_low'][0].split('/')[7]+'/pred_high_' +batch_data['dcm_low'][0].split('/')[-1])        
        save_dicom(batch_data['dcm_high'][0], input_high, test_high2low_save_folder+batch_data['dcm_high'][0].split('/')[7]+'/gt_high_' +batch_data['dcm_high'][0].split('/')[-1])
        save_dicom(batch_data['dcm_high'][0], output_low, test_high2low_save_folder+batch_data['dcm_high'][0].split('/')[7]+'/pred_low_' +batch_data['dcm_high'][0].split('/')[-1])
        
        plt.figure(figsize=(12,12))
        plt.subplot(121)
        plt.imshow(output_low.squeeze())
        plt.subplot(122)
        plt.imshow(output_high.squeeze())
        plt.show()
#         break
#         # 0과 1로 안전빵
#         input_low = np.clip(input_low, a_min=0, a_max=1)
#         input_high = np.clip(input_high, a_min=0, a_max=1)
        
#         output_high = np.clip(output_high, a_min=0, a_max=1)        
#         output_low = np.clip(output_low, a_min=0, a_max=1)
        